# Neuronal Ca²⁺ Activity — Minimal ΔF/F Pipeline Demo (Toy)
This notebook reproduces the same toy workflow as `calcium_peaks_demo.py`.
**Synthetic data only.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## 1) Simulate a calcium-like fluorescence trace

In [ ]:
np.random.seed(0)
frames = 1000
baseline_value = 50.0
noise_sigma = 2.0

raw_trace = np.random.normal(baseline_value, noise_sigma, frames)

event_frames = [100, 250, 500, 750, 900]      # 0-based indices
event_amplitudes = [52, 58, 51, 42, 47]       # added intensity
for idx, amp in zip(event_frames, event_amplitudes):
    raw_trace[idx] += amp

raw_trace[:10], raw_trace.mean()


## 2) Compute ΔF/F

In [ ]:
F0 = float(np.mean(raw_trace[:100]))
dff = (raw_trace - F0) / F0

F0, dff[:10]


## 3) Detect events (toy threshold rule)

In [ ]:
threshold = float(np.mean(dff) + 1.5 * np.std(dff))
peak_indices = np.where(dff > threshold)[0]

threshold, peak_indices[:20], len(peak_indices)


## 4) Export CSVs + QC plot

In [ ]:
df = pd.DataFrame({
    "Frame": np.arange(1, frames + 1),
    "Raw": raw_trace,
    "DeltaF/F": dff
})
df.head()


In [ ]:
df.to_csv("raw_trace_dfF.csv", index=False)

events = pd.DataFrame({
    "Frame": peak_indices + 1,   # 1-based
    "Value": dff[peak_indices]
})
events.to_csv("events.csv", index=False)

plt.figure(figsize=(10, 4))
plt.plot(dff, label="ΔF/F")
plt.scatter(peak_indices, dff[peak_indices], s=18, label="Detected events")
plt.axhline(y=0, linestyle="--", linewidth=0.8)
plt.xlabel("Frame")
plt.ylabel("ΔF/F")
plt.title("Simulated Ca²⁺ Trace with Detected Peaks (Toy Demo)")
plt.legend()
plt.tight_layout()
plt.savefig("trace_qc.png", dpi=200)
plt.show()

print("Saved: raw_trace_dfF.csv, events.csv, trace_qc.png")


## Notes / limitations
- This toy detector flags every frame above a simple threshold.
- Real calcium event detection usually adds merging/refractory logic, baseline drift handling, etc.